In [1]:
gameFolder = "DOA++-MAME"
#gameFolder = "SFIII-MAME"
#gameFolder = "UMK3-MAME"
#gameFolder = "TEKTAG-MAME"

In [2]:
import sys, os
import time
timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../../games',gameFolder))   

tensorBoardFolder = "./ppo2_TB_" + gameFolder + "/"
modelFolder = "ppo2_Model_" + gameFolder

In [3]:
from makeDiambraEnv import *

%load_ext tensorboard

from stable_baselines.common.policies import CnnPolicy
from stable_baselines import PPO2

In [ ]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../customMAME/"
diambraKwargs["player"] = "P1"
diambraKwargs["frame_ratio"] = 3
#diambraKwargs["render"] =True
#diambraKwargs["throttle"] = False
#diambraKwargs["sound"] = False 
#diambraKwargs["character"] ="Random"
diambraKwargs["character"] = "Kasumi"

wrapperKwargs = {"frame_stack": True}
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["scale"] = True
wrapperKwargs["hw_obs_resize"] = [128, 128]

numEnv=4

env = make_diambra_env(diambraMame, env_prefix="Train", num_env=numEnv, seed=timeDepSeed, 
                       diambra_kwargs = diambraKwargs, wrapper_kwargs = wrapperKwargs)

# OR 
#env = make_diambra_env(diambraMame, num_env=2, seed=0, diambra_kwargs = diambraKwargs)
# Frame-stacking with 4 frames
#env = VecFrameStack(env, n_stack=4)

Env_id =  Train0


In [ ]:
model = PPO2(CnnPolicy, env, verbose=1, tensorboard_log=tensorBoardFolder)

In [ ]:
# Load the trained agent
#model = PPO2.load(modelFolder)

In [ ]:
# Train the agent
time_steps = 10000000
model.learn(total_timesteps=time_steps)

In [ ]:
# Save the agent
model.save(modelFolder)

In [ ]:
%tensorboard --logdir ./tensorBoardFolder/

In [ ]:
observation = env.reset()

while True:

    actions = []
    for idx in range(numEnv):
        actions.append(env.action_space.sample())

    observation, reward, done, info = env.step(actions)

    print("Frames shape:", observation.shape)
    print("Reward:", reward)
    #print("Fighting = ", info["fighting"])
    #print("Rewards = ", info["rewards"])
    #print("HealthP1 = ", info["healthP1"])
    #print("HealthP2 = ", info["healthP2"])
    #print("PositionP1 = ", info["positionP1"])
    #print("PositionP2 = ", info["positionP2"])
    #print("WinP1 = ", info["winsP1"])
    #print("WinP2 = ", info["winsP2"])
    print("Done = ", done)

    if np.any(done):
        print("Resetting Env")
        observation = env.reset()
        
env.close()

In [ ]:
# Enjoy trained agent
eval_env = make_atari_env('PongNoFrameskip-v4', num_env=1, seed=0,  wrapper_kwargs = {"frame_stack": True})
obs = eval_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=False)
    obs, reward, done, info = eval_env.step(action)
    eval_env.render()
    if done:
        obs = eval_env.reset()
eval_env.close()